In [ ]:
class apple():
    def __init__(self):
        self.a = 'b'
        self.ou = base()
    def oo(self):
        self.ou.oput(self)

class base():
    def __init_(self):
        pass
    def oput(self,m):
        print(m.a)

a = apple()
a.oo()

In [ ]:
%load_ext autoreload
%autoreload 2

import osj
import sys
sys.path.append(os.getcwd())

import torch
from torch import nn
import numpy as np
from matplotlib import pyplot as plt
from torchvision.utils import make_grid

from BigGAN.gan_load import make_big_gan
from utils.utils import to_image
from gan_mask_gen import MaskGenerator, MaskSynthesizing
from train_segmentation import TrainParams
from test_frame import TestParams_jupyter
from visualization import draw_with_mask
from postprocessing import connected_components_filter
%matplotlib inline

## Visulization of Segment map and CAM map

In [ ]:
from random import randint
from UNet.unet_cam import UNet_cam_v5 as UNet_cam
from data import SegFileDataset
from postprocessing import SegmentationInference, Threshold
from metrics import F_max, IoU, accuracy, model_metrics,Localization
from visualization import overlayed
from data import TestDataset
import ipdb
from postprocessing import CAMInference

import cv2
#模型准备
%matplotlib inline
unet_load_paths = '/home/qxy/Desktop/BigGan/weight_result/results/trained_weight/000122_result.pth'
unet = UNet_cam().eval()
unet.load_state_dict(torch.load(unet_load_paths))
unet.eval()

batch_size = 1
param = TestParams_jupyter()

# target_ds = TestDataset(param.image_root_dir,param.image_property_dir)
target_ds = SegFileDataset(param,crop=False,size=128)

segmentation_dl = torch.utils.data.DataLoader(target_ds, batch_size, shuffle=False)

img,label = target_ds[3000]

mask_pre = CAMInference(unet)(img.unsqueeze(0),label=200)

plt.figure(figsize=(3,1),dpi=450)
# image = make_grid(torch.clamp(mask_pre, -1, 1), nrow=batch_size, padding=5)
# print(mask_pre.shape)
plt.subplot(1,2,1)
plt.axis('off')

# image=to_image(image,True)
# plt.imshow(image)
# # image=to_image(imgs_grid,True)
image = (mask_pre - mask_pre.min()) / (mask_pre.max() - mask_pre.min())
image = (255 * image).squeeze().cpu().detach().numpy()
image = image.astype(np.uint8)

image1=cv2.applyColorMap(image, cv2.COLORMAP_JET)
print(image1.shape)

image1 = image1[:,:,[2,1,0]]
plt.imshow(image1)
plt.subplot(1,2,2)
images = to_image(img)
plt.imshow(images)
plt.subplots_adjust(wspace=0, hspace=0)

In [ ]:
from random import randint
from UNet.unet_cam import UNet_cam_v5 as UNet_cam
from data import SegFileDataset
from postprocessing import SegmentationInference, Threshold
from metrics import F_max, IoU, accuracy, model_metrics,Localization
from visualization import overlayed
from data import TestDataset
import ipdb
from postprocessing import CAMInference
import cv2
from torchvision.utils import make_grid
from visualization import overlayed
from utils.utils import to_image
batch_size = 1
param = TestParams_jupyter()

unet_load_paths = '/home/qxy/Desktop/BigGan/weight_result/results/trained_weight/000122_result.pth'
unet = UNet_cam().eval()
unet.load_state_dict(torch.load(unet_load_paths))
unet.eval()

target_ds = SegFileDataset(param,crop=False,size=None)
img,mask = target_ds[789]
img = img.unsqueeze(0)
mask_prediction = SegmentationInference(unet, resize_to=128)(img) > 0.8

plt.figure(figsize=(1, 3.), dpi=450)
imgs_grid = make_grid(torch.clamp(img, -1, 1), nrow=batch_size, padding=5)
plt.imshow(to_image(overlayed(img, mask_prediction), True))

## Visulization of CAM classification model

In [ ]:
from random import randint
from UNet.unet_cam import UNet_cam_v4 as UNet_cam
from data import SegFileDataset
from postprocessing import SegmentationInference, Threshold
from metrics import F_max, IoU, accuracy, model_metrics,Localization
from visualization import overlayed
from data import TestDataset
import ipdb

unet_load_paths = '/home/qxy/Desktop/BigGan/weight_result/results/trained_weight/000103_result.pth'
unet = UNet_cam().eval()
unet.load_state_dict(torch.load(unet_load_paths))
unet.eval()

batch_size = 1
param = TestParams_jupyter()

target_ds = TestDataset(param.image_root_dir,param.image_property_dir)
segmentation_dl = torch.utils.data.DataLoader(target_ds, batch_size, shuffle=False)

In [ ]:
from postprocessing import CAMInference
import cv2
%matplotlib inline
img,label = target_ds[300]

mask_pre = CAMInference(unet)(img.unsqueeze(0),label)
print(mask_pre.shape)
plt.figure(figsize=(3,1),dpi=450)
# image = make_grid(torch.clamp(mask_pre, -1, 1), nrow=batch_size, padding=5)
# print(mask_pre.shape)
plt.subplot(1,2,1)
plt.axis('off')

# image=to_image(image,True)
# plt.imshow(image)
# # image=to_image(imgs_grid,True)
image = (mask_pre - mask_pre.min()) / (mask_pre.max() - mask_pre.min())
image = (255 * image).squeeze().cpu().detach().numpy()
image = image.astype(np.uint8)
print(image.shape)
image1=cv2.applyColorMap(image, cv2.COLORMAP_JET)
plt.imshow(image1)
plt.subplot(1,2,2)
images = to_image(img)
plt.imshow(images)
plt.subplots_adjust(wspace=0, hspace=0)

In [ ]:
images = to_image(img)
#print(images.shape)
plt.imshow(images)

In [ ]:
from random import randint
from UNet.unet_model import UNet
from data import SegFileDataset
from postprocessing import SegmentationInference, Threshold
from metrics import F_max, IoU, accuracy, model_metrics,Localization
from visualization import overlayed

import ipdb
unet_load_paths = '/home/qxy/Desktop/BigGan/weight_result/results/trained_weight/000008_result.pth'
unet = UNet().cuda().eval()
unet.load_state_dict(torch.load(unet_load_paths, map_location='cpu'))
unet.cuda().eval()


batch_size = 1
# CUB_imgs_dir = '/home/qxy/Desktop/datasets/CUB/data'
# CUB_masks_dir = '/home/qxy/Desktop/datasets/CUB/segmentation'
# image_prop = '/home/qxy/Desktop/datasets/CUB/data_annotation/'+\
#             'CUB_WSOL/test_list.json'
param = TestParams_jupyter()

target_ds = SegFileDataset(param,crop=False, size=None)
segmentation_dl = torch.utils.data.DataLoader(target_ds, batch_size, shuffle=False)

In [ ]:
print('Raw model:\n{}'.format(
    model_metrics(Threshold(unet, resize_to=128), \
                  segmentation_dl, stats=(Localization,) )))
# print('Thresholded:\n{}'.format(
#     model_metrics(Threshold(unet, thr=0.5, resize_to=128), segmentation_dl, stats=(IoU, accuracy))))

In [ ]:
a = 'localization'
print(a.__name__)

# Synthetic data

In [ ]:
%load_ext autoreload
%autoreload 2
import torch
from BigGAN.gan_load import make_big_gan
from utils.utils import to_image
from gan_mask_gen import MaskGenerator, MaskSynthesizing
from train_segmentation import TrainParams_jupyter
from visualization import draw_with_mask
from postprocessing import connected_components_filter
import ipdb
import numpy as np
%matplotlib inline
biggan_path = '/home/qxy/Desktop/BigGan/weight_result/weights/BigBiGAN_x1.pth'
bg_path = '/home/qxy/Desktop/BigGan/weight_result/weights/bg_direction.pth'
G = make_big_gan(biggan_path).cuda().eval()
bg_direction = torch.load(bg_path).cuda()

In [ ]:

params = TrainParams_jupyter(batch_size=16,)
# z = torch.randn([params.batch_size, G.dim_z]).cuda()
embedding_path = '/home/qxy/Desktop/BigGan/weight_result/embeddings/BigBiGAN_ImageNet_z.npy'
embedding_cls_path = '/home/qxy/Desktop/BigGan/weight_result/embeddings/BigBiGAN_CUB_WSOL_train_z_cls.npy'
z = torch.from_numpy(np.load(embedding_path)).cuda()[[x for x in range(0, 974445)]]
z_cls = torch.from_numpy(np.load(embedding_cls_path)).cuda()[[0,100,1000,500,1500,2500,300,450]]#[0:30]

mg = MaskGenerator(G, bg_direction, params, zs=z,zs_cls=z_cls,
                       mask_postprocessing=(connected_components_filter,))
mg.cuda()

img, ref = mg()
draw_with_mask(img, [ref.unsqueeze(1)], names=[''], horizontal=False)
# print(index)

# Evaluation

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append(os.getcwd())
import torch
from torch import nn
import numpy as np
from matplotlib import pyplot as plt
from torchvision.utils import make_grid

from BigGAN.gan_load import make_big_gan
from utils.utils import to_image
from gan_mask_gen import MaskGenerator, MaskSynthesizing
from train_segmentation import TrainParams
from test_frame import TestParams_jupyter
from visualization import draw_with_mask
from postprocessing import connected_components_filter

from random import randint
from UNet.unet_cam import UNet_cam_v4 as UNet_cam
from data import SegFileDataset
from postprocessing import SegmentationInference, Threshold
from metrics import F_max, IoU, accuracy, model_metrics,Localization
from visualization import overlayed
from data import TestDataset


import ipdb
unet_load_paths = '/home/qxy/Desktop/BigGan/weight_result/results/trained_weight/000103_result.pth'
unet = UNet_cam().eval()
unet.load_state_dict(torch.load(unet_load_paths))
unet.eval()


batch_size = 1
# CUB_imgs_dir = '/home/qxy/Desktop/datasets/CUB/data'
# CUB_masks_dir = '/home/qxy/Desktop/datasets/CUB/segmentation'
# image_prop = '/home/qxy/Desktop/datasets/CUB/data_annotation/'+\
#             'CUB_WSOL/test_list.json'
param = TestParams_jupyter()

target_ds = SegFileDataset(param,crop=False, size=None)
segmentation_dl = torch.utils.data.DataLoader(target_ds, batch_size, shuffle=False)

In [ ]:
%matplotlib inline
ct = 400
scale =5
mask_prediction = []
mask = []
with torch.no_grad():
    img = []
    label = []
    #ipdb.set_trace()

    for i in range(ct,ct+5):
        #ipdb.set_trace()
        #img[i-ct], mask[i-ct] = target_ds[i]#[randint(0, len(target_ds))]
        img_tmp,mask_tmp = target_ds[i]
        img.append(img_tmp)
        mask.append(mask_tmp)
        img_tmp, mask_tmp = img[i-ct].unsqueeze(0), mask[i-ct].unsqueeze(0)
        print(img_tmp.shape)
        ipdb.set_trace()
        mask_prediction.append( SegmentationInference(unet, resize_to=128)(img_tmp) > 0.5 )
    ct+=5

plt.figure(figsize=(3, 5.), dpi=450)
for i in range(scale):
    imgs_grid = make_grid(torch.clamp(img[i], -1, 1), nrow=batch_size, padding=5)

    print(i)
    plt.subplot(3,6,1+i*3)
    plt.axis('off')
    plt.imshow(to_image(imgs_grid, True))
    plt.subplot(3,6,2+i*3)
    plt.axis('off')
    plt.title('Ground truth',fontsize=5)
    plt.imshow(to_image(overlayed(img[i], mask[i], channel=0), True))
    plt.subplot(3,6,3+i*3)
    plt.axis('off')
    plt.title('Prediction',fontsize=5)
    plt.imshow(to_image(overlayed(img[i], mask_prediction[i]), True))

    plt.subplots_adjust(wspace=0, hspace=0)

In [ ]:
from postprocessing import CAMInference
import cv2
%matplotlib inline
img,label = target_ds[3006]

mask_pre = CAMInference(unet)(img.unsqueeze(0),label)
print(mask_pre.shape)
plt.figure(figsize=(3,1),dpi=450)
# image = make_grid(torch.clamp(mask_pre, -1, 1), nrow=batch_size, padding=5)
# print(mask_pre.shape)
plt.subplot(1,1,1)
plt.axis('off')

# image=to_image(image,True)
# plt.imshow(image)
# # image=to_image(imgs_grid,True)
image = (mask_pre - mask_pre.min()) / (mask_pre.max() - mask_pre.min())
image = (255 * image).squeeze().cpu().detach().numpy()
image = image.astype(np.uint8)
print(image.shape)
image1=cv2.applyColorMap(image, cv2.COLORMAP_JET)

plt.imshow(image1)
plt.subplots_adjust(wspace=0, hspace=0)

In [ ]:
#t = 500
scale =5
with torch.no_grad():
    img = []
    label = []
    CAMs = []
    #ipdb.set_trace()

    for i in range(ct,ct+5):
        #ipdb.set_trace()
        #img[i-ct], mask[i-ct] = target_ds[i]#[randint(0, len(target_ds))]
        img_tmp,label_tmp = target_ds[i]
        img.append(img_tmp)
        label.append(label_tmp)
        img_tmp, label_tmp = img[i-ct].cuda().unsqueeze(0), label[i-ct].cuda().unsqueeze(0)
        mask_prediction.append( SegmentationInference(unet, resize_to=128)(img_tmp) > 0.5 )
    ct+=5

plt.figure(figsize=(3, 5.), dpi=450)
for i in range(scale):
    imgs_grid = make_grid(torch.clamp(img[i], -1, 1), nrow=batch_size, padding=5)

    print(i)
    plt.subplot(3,6,1+i*3)
    plt.axis('off')
    plt.imshow(to_image(imgs_grid, True))
    plt.subplot(3,6,2+i*3)
    plt.axis('off')
    plt.title('Ground truth',fontsize=5)
    plt.imshow(to_image(overlayed(img[i], mask[i], channel=0), True))
    plt.subplot(3,6,3+i*3)
    plt.axis('off')
    plt.title('Prediction',fontsize=5)
    plt.imshow(to_image(overlayed(img[i], mask_prediction[i]), True))

    plt.subplots_adjust(wspace=0, hspace=0)

In [ ]:
with torch.no_grad():
    img, mask = target_ds[randint(0, len(target_ds))]
    img, mask = img.cuda().unsqueeze(0), mask.cuda().unsqueeze(0)
    mask_prediction = SegmentationInference(unet, resize_to=128)(img) > 0.5


imgs_grid = make_grid(torch.clamp(img, -1, 1), nrow=batch_size, padding=5)
plt.figure(figsize=(9, 15.), dpi=250)

plt.subplot(131)
plt.axis('off')
plt.imshow(to_image(imgs_grid, True))
plt.axis('off')
plt.title('Ground truth')
plt.imshow(to_image(overlayed(img, mask, channel=0), True))
plt.subplot(133)
plt.axis('off')
plt.title('Prediction')
plt.imshow(to_image(overlayed(img, mask_prediction), True))

plt.subplots_adjust(wspace=0, hspace=0)

In [ ]:
print('Raw model:\n{}'.format(
    model_metrics(SegmentationInference(unet, resize_to=128), segmentation_dl, stats=(F_max,))))
print('Thresholded:\n{}'.format(
    model_metrics(Threshold(unet, thr=0.5, resize_to=128), segmentation_dl, stats=(IoU, accuracy))))

In [ ]:
words = 'asodijofijsdlfna;weqxyxyyxxxxyyyxqyqqyqqxxy'
print(words.replace('qxy','---'))